from example https://github.com/reichenbch/RAG-examples/blob/main/LangChain%20LLamaIndex%20RAG.ipynb

In [1]:
import sys
sys.path.insert(0, '/home/sulcan/Documents/ipac-logbook/code/')

In [11]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.vectorstores import FAISS
from hugginface_wrapper import MistralLLM
from langchain.chains.question_answering import load_qa_chain
import pandas as pd

In [3]:
df = pd.read_csv('/home/sulcan/Documents/malapa/data/abstracts.csv')
abstracts = list(df['Content'])

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") 
vector_store = FAISS.from_texts(abstracts, embeddings)
vector_store.save_local("../data/faiss_doc_idx")
docs = vector_store.similarity_search("What is anomaly detection?")

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

https://python.langchain.com/docs/integrations/chat/mistralai

In [5]:
llm = MistralLLM(max_new_tokens = 128)

'''
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="knock knock")]
llm.invoke(messages)
'''

'\nfrom langchain_core.messages import HumanMessage\nmessages = [HumanMessage(content="knock knock")]\nllm.invoke(messages)\n'

In [6]:
# https://blog.gopenai.com/unveiling-retrieval-qa-and-load-qa-chain-for-langchain-question-answering-42de13c1de84
chain = load_qa_chain(llm, chain_type="refine")

In [18]:
language = "English"
template = """I want you to act as a question answering bot which uses the context mentioned and answer in a concise manner and doesn't make stuff up.
            You will answer question based on the context - {context}.
            You will create content in """ + str(language) + """language.
            Question: {question}
            Answer:
            """

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vector_store.as_retriever(), 
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


In [19]:
result = qa_chain({"query": "Is there someone talking about anomaly detection"})

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [20]:
print(result['result'])

[INST] I want you to act as a question answering bot which uses the context mentioned and answer in a concise manner and doesn't make stuff up.
            You will answer question based on the context - Machine Learning (ML) has gained significant prominence in the field of engineering due to its adaptability and versatility. An example of its practical application is in anomaly detection, which serves the fundamental purpose of providing a binary response to the question, "Has an issue arisen?". Most machine learning and anomaly detection projects strive to provide generalisable solutions that are robust to changes within systems. This is of particular importance when components that are subject to anomalies are upgraded; the behaviour and values in the system remain unchanged, but the data resolution may be higher, and anomalies may manifest themselves in slightly different ways. An ideal ML solution would be able to detect anomalies as accurately in this new system as in the old on